## Lab 4: Deploy to Production - Use AgentCore Runtime with Observability

### Overview

In Lab 3 we scaled our Customer Support Agent by centralizing tools through AgentCore Gateway with secure authentication. Now we'll complete the production journey by deploying our agent to AgentCore Runtime with comprehensive observability. This will transform our prototype into a production-ready system that can handle real-world traffic with full monitoring and automatic scaling.

[Amazon Bedrock AgentCore Runtime](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/agents-tools-runtime.html) is a secure, fully managed runtime that empowers organizations to deploy and scale AI agents in production, regardless of framework, protocol, or model choice. It provides enterprise-grade reliability, automatic scaling, and comprehensive monitoring capabilities.

**Workshop Journey:**

- **Lab 1 (Done):** Create Agent Prototype - Built a functional customer support agent
- **Lab 2 (Done):** Enhance with Memory - Added conversation context and personalization
- **Lab 3 (Done):** Scale with Gateway & Identity - Shared tools across agents securely
- **Lab 4 (Current):** Deploy to Production - Used AgentCore Runtime with observability
- **Lab 5:** Build User Interface - Create a customer-facing application

### Why AgentCore Runtime & Production Deployment Matter

Current State (Lab 1-3): Agent runs locally with centralized tools but faces production challenges:

- Agent runs locally in a single session
- No comprehensive monitoring or debugging capabilities
- Cannot handle multiple concurrent users reliably

After this lab, we will have a production-ready agent infrastructure with:

- Serverless auto-scaling to handle variable demand
- Comprehensive observability with traces, metrics, and logging
- Enterprise reliability with automatic error recovery
- Secure deployment with proper access controls
- Easy management through AWS console and APIs and support for real-world production workloads.


### Adding comprehensive observability with AgentCore Observability

Additionally, AgentCore Runtime integrates seamlessly with [AgentCore Observability](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/observability.html) to provide full visibility into your agent's behavior in production. AgentCore Observability automatically captures traces, metrics, and logs from your agent interactions, tool usage, and memory access patterns. In this lab we will see how AgentCore Runtime integrates with CloudWatch GenAI Observability to provide comprehensive monitoring and debugging capabilities.

For request tracing, AgentCore Observability captures the complete conversation flow including tool invocations, memory retrievals, and model interactions. For performance monitoring, it tracks response times, success rates, and resource utilization to help optimize your agent's performance.

During the observability flow, AgentCore Runtime automatically instruments your agent code and sends telemetry data to CloudWatch. You can then use CloudWatch dashboards and GenAI Observability features to analyze patterns, identify bottlenecks, and troubleshoot issues in real-time.

### Architecture for Lab 4
<div style="text-align:left"> 
    <img src="images/architecture_lab4_runtime.png" width="75%"/> 
</div>

*Agent now runs in AgentCore Runtime with full observability through CloudWatch, serving production traffic with auto-scaling and comprehensive monitoring. Memory and Gateway integrations from previous labs remain fully functional in the production environment.*

### Key Features

- **Serverless Agent Deployment:** Transform your local agent into a scalable production service using AgentCore Runtime with minimal code changes
- **Comprehensive Observability:** Full request tracing, performance metrics, and debugging capabilities through CloudWatch GenAI Observability

### Prerequisites

- Python 3.12+
- AWS account with appropriate permissions
- Docker, Finch or Podman installed and running
- Amazon Bedrock AgentCore SDK
- Strands Agents framework

**Note**: You MUST enable [CloudWatch Transaction Search](https://docs.aws.amazon.com/AmazonCloudWatch/latest/monitoring/Enable-TransactionSearch.html) to be able to see AgentCore Observability traces in CloudWatch.


### Step 1: Import Required Libraries

In [1]:
# Import required libraries
import os
import json
import boto3
from strands import Agent
from strands.models import BedrockModel
from lab_helpers.lab2_memory import create_or_get_memory_resource

create_or_get_memory_resource()  # Just in case the memory lab wasn't executed

'CustomerSupportMemory-EtWANNBq0n'

### Step 2: Preparing Your Agent for AgentCore Runtime

#### Creating the Runtime-Ready Agent

Let's first define the necessary AgentCore Runtime components via Python SDK within our previous local agent implementation.

Observe the `#### AGENTCORE RUNTIME - LINE i ####` comments below to see where is the relevant deployment code added. You'll find 4 such lines that prepare the runtime-ready agent:

1. Import the Runtime App with `from bedrock_agentcore.runtime import BedrockAgentCoreApp`
2. Initialize the App with `app = BedrockAgentCoreApp()`
3. Decorate our invocation function with `@app.entrypoint`
4. Let AgentCore Runtime control the execution with `app.run()`


In [2]:
%%writefile ./lab_helpers/lab4_runtime.py
from bedrock_agentcore.runtime import (
    BedrockAgentCoreApp,
)  #### AGENTCORE RUNTIME - LINE 1 ####
from strands import Agent
from strands.models import BedrockModel
from scripts.utils import get_ssm_parameter
from lab_helpers.lab1_strands_agent import (
    get_return_policy,
    get_product_info,
    SYSTEM_PROMPT,
    MODEL_ID,
)

from lab_helpers.lab2_memory import (
    CustomerSupportMemoryHooks,
    memory_client,
    ACTOR_ID,
    SESSION_ID,
)

# Lab1 import: Create the Bedrock model
model = BedrockModel(model_id=MODEL_ID)

# Lab2 import : Initialize memory via hooks
memory_id = get_ssm_parameter("/app/customersupport/agentcore/memory_id")
memory_hooks = CustomerSupportMemoryHooks(
    memory_id, memory_client, ACTOR_ID, SESSION_ID
)

# Create the agent with all customer support tools
agent = Agent(
    model=model,
    tools=[get_return_policy, get_product_info],
    system_prompt=SYSTEM_PROMPT,
    hooks=[memory_hooks],
)

# Initialize the AgentCore Runtime App
app = BedrockAgentCoreApp()  #### AGENTCORE RUNTIME - LINE 2 ####


@app.entrypoint  #### AGENTCORE RUNTIME - LINE 3 ####
def invoke(payload):
    """AgentCore Runtime entrypoint function"""
    user_input = payload.get("prompt", "")

    # Invoke the agent
    response = agent(user_input)
    return response.message["content"][0]["text"]


if __name__ == "__main__":
    app.run()  #### AGENTCORE RUNTIME - LINE 4 ####

Overwriting ./lab_helpers/lab4_runtime.py


#### What happens behind the scenes?

When you use `BedrockAgentCoreApp`, it automatically:

- Creates an HTTP server that listens on port 8080
- Implements the required `/invocations` endpoint for processing requests
- Implements the `/ping` endpoint for health checks
- Handles proper content types and response formats
- Manages error handling according to AWS standards


### Step 3: Deploying to AgentCore Runtime

Now let's deploy our agent to AgentCore Runtime using the [AgentCore Starter Toolkit](https://github.com/aws/bedrock-agentcore-starter-toolkit).

#### Configure the Secure Runtime Deployment (AgentCore Runtime + AgentCore Identity)

First we will use our starter toolkit to configure the AgentCore Runtime deployment with an entrypoint, the execution role we will create and a requirements file. We will also configure the identity authorization using an Amazon Cognito user pool and we will configure the starter kit to auto create the Amazon ECR repository on launch.

During the configure step, your docker file will be generated based on your application code

<div style="text-align:left"> 
    <img src="images/configure.png" width="75%"/> 
</div>

**Note**: The Cognito access_token is valid for 2 hours only. If the access_token expires you can vend another access_token by using the `reauthenticate_user` method.


In [3]:
from lab_helpers.utils import setup_cognito_user_pool, reauthenticate_user

print("Setting up Amazon Cognito user pool...")
cognito_config = (
    setup_cognito_user_pool()
)  # You'll get your bearer token from this output cell.
print("Cognito setup completed ✓")

Setting up Amazon Cognito user pool...
{'UserPoolId': 'us-east-1_8eROuquri', 'ClientName': 'MCPServerPoolClient', 'ClientId': '6alrcnckui47t036qvon48p4a0', 'ClientSecret': '1qrjbpvqr2r1gd5oecuvpqnf50tge4r83qqag3ld1dcqf269oh1r', 'LastModifiedDate': datetime.datetime(2025, 8, 16, 7, 15, 42, 204000, tzinfo=tzlocal()), 'CreationDate': datetime.datetime(2025, 8, 16, 7, 15, 42, 204000, tzinfo=tzlocal()), 'RefreshTokenValidity': 30, 'TokenValidityUnits': {}, 'ExplicitAuthFlows': ['ALLOW_USER_PASSWORD_AUTH', 'ALLOW_USER_SRP_AUTH', 'ALLOW_REFRESH_TOKEN_AUTH'], 'AllowedOAuthFlowsUserPoolClient': False, 'EnableTokenRevocation': True, 'EnablePropagateAdditionalUserContextData': False, 'AuthSessionValidity': 3}
Pool id: us-east-1_8eROuquri
Discovery URL: https://cognito-idp.us-east-1.amazonaws.com/us-east-1_8eROuquri/.well-known/openid-configuration
Client ID: 6alrcnckui47t036qvon48p4a0
Bearer Token: eyJraWQiOiJZa1Q3cTJKZUllNFVPSld1TEJ3NHZCVlwvZVAxVzdwbW5HMnZcL1g2eWl4cFE9IiwiYWxnIjoiUlMyNTYifQ.eyJz

In [4]:
from bedrock_agentcore_starter_toolkit import Runtime
from lab_helpers.utils import create_agentcore_runtime_execution_role

# Initialize the runtime toolkit
boto_session = boto3.session.Session()
region = boto_session.region_name

execution_role_arn = create_agentcore_runtime_execution_role()

agentcore_runtime = Runtime()

# Configure the deployment
response = agentcore_runtime.configure(
    entrypoint="lab_helpers/lab4_runtime.py",
    execution_role=execution_role_arn,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name="customer_support_agent",
    authorizer_configuration={
        "customJWTAuthorizer": {
            "allowedClients": [cognito_config.get("client_id")],
            "discoveryUrl": cognito_config.get("discovery_url"),
        }
    },
)

print("Configuration completed:", response)

✅ Created IAM role: CustomerSupportAssistantBedrockAgentCoreRole-us-east-1
Role ARN: arn:aws:iam::057716757052:role/CustomerSupportAssistantBedrockAgentCoreRole-us-east-1
✅ Created policy: CustomerSupportAssistantBedrockAgentCorePolicy-us-east-1
✅ Attached policy to role
Policy ARN: arn:aws:iam::057716757052:policy/CustomerSupportAssistantBedrockAgentCorePolicy-us-east-1


Entrypoint parsed: file=/home/ubuntu/Self-Study-Generative-AI/lab/17_bedrock_agent_core/01-tutorials/07-AgentCore-E2E/lab_helpers/lab4_runtime.py, bedrock_agentcore_name=lab4_runtime
Configuring BedrockAgentCore agent: customer_support_agent


⚠️  [WARNING] Platform mismatch: Current system is 'linux/amd64' but Bedrock AgentCore requires 'linux/arm64'.
For deployment options and workarounds, see: 
https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/getting-started-custom.html

Generated .dockerignore
Generated Dockerfile: /home/ubuntu/Self-Study-Generative-AI/lab/17_bedrock_agent_core/01-tutorials/07-AgentCore-E2E/Dockerfile
Generated .dockerignore: /home/ubuntu/Self-Study-Generative-AI/lab/17_bedrock_agent_core/01-tutorials/07-AgentCore-E2E/.dockerignore
Setting 'customer_support_agent' as default agent
Bedrock AgentCore configured: /home/ubuntu/Self-Study-Generative-AI/lab/17_bedrock_agent_core/01-tutorials/07-AgentCore-E2E/.bedrock_agentcore.yaml


Configuration completed: config_path=PosixPath('/home/ubuntu/Self-Study-Generative-AI/lab/17_bedrock_agent_core/01-tutorials/07-AgentCore-E2E/.bedrock_agentcore.yaml') dockerfile_path=PosixPath('/home/ubuntu/Self-Study-Generative-AI/lab/17_bedrock_agent_core/01-tutorials/07-AgentCore-E2E/Dockerfile') dockerignore_path=PosixPath('/home/ubuntu/Self-Study-Generative-AI/lab/17_bedrock_agent_core/01-tutorials/07-AgentCore-E2E/.dockerignore') runtime='Docker' region='us-east-1' account_id='057716757052' execution_role='arn:aws:iam::057716757052:role/CustomerSupportAssistantBedrockAgentCoreRole-us-east-1' ecr_repository=None auto_create_ecr=True


#### Launch the Agent

Now let's launch our agent to AgentCore Runtime. This will create an AWS CodeBuild pipeline, the Amazon ECR repository and the AgentCore Runtime components.

<div style="text-align:left"> 
    <img src="images/launch.png" width="100%"/> 
</div>

In [5]:
# Launch the agent (this will build and deploy the container)
from lab_helpers.utils import put_ssm_parameter

launch_result = agentcore_runtime.launch()
print("Launch completed:", launch_result.agent_arn)

agent_arn = put_ssm_parameter(
    "/app/customersupport/agentcore/runtime_arn", launch_result.agent_arn
)

🚀 CodeBuild mode: building in cloud (RECOMMENDED - DEFAULT)
   • Build ARM64 containers in the cloud with CodeBuild
   • No local Docker required
💡 Available deployment modes:
   • runtime.launch()                           → CodeBuild (current)
   • runtime.launch(local=True)                 → Local development
   • runtime.launch(local_build=True)           → Local build + cloud deploy (NEW)
Starting CodeBuild ARM64 deployment for agent 'customer_support_agent' to account 057716757052 (us-east-1)
Starting CodeBuild ARM64 deployment for agent 'customer_support_agent' to account 057716757052 (us-east-1)
Setting up AWS resources (ECR repository, execution roles)...
Getting or creating ECR repository for agent: customer_support_agent
✅ ECR repository available: 057716757052.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-customer_support_agent
Using execution role from config: arn:aws:iam::057716757052:role/CustomerSupportAssistantBedrockAgentCoreRole-us-east-1


Repository doesn't exist, creating new ECR repository: bedrock-agentcore-customer_support_agent


✅ Execution role validation passed: arn:aws:iam::057716757052:role/CustomerSupportAssistantBedrockAgentCoreRole-us-east-1
Preparing CodeBuild project and uploading source...
Getting or creating CodeBuild execution role for agent: customer_support_agent
Role name: AmazonBedrockAgentCoreSDKCodeBuild-us-east-1-aa96c3a063
CodeBuild role doesn't exist, creating new role: AmazonBedrockAgentCoreSDKCodeBuild-us-east-1-aa96c3a063
Creating IAM role: AmazonBedrockAgentCoreSDKCodeBuild-us-east-1-aa96c3a063
✓ Role created: arn:aws:iam::057716757052:role/AmazonBedrockAgentCoreSDKCodeBuild-us-east-1-aa96c3a063
Attaching inline policy: CodeBuildExecutionPolicy to role: AmazonBedrockAgentCoreSDKCodeBuild-us-east-1-aa96c3a063
✓ Policy attached: CodeBuildExecutionPolicy
Waiting for IAM role propagation...
CodeBuild execution role creation complete: arn:aws:iam::057716757052:role/AmazonBedrockAgentCoreSDKCodeBuild-us-east-1-aa96c3a063
Using .dockerignore with 44 patterns
Uploaded source to S3: customer_su

Launch completed: arn:aws:bedrock-agentcore:us-east-1:057716757052:runtime/customer_support_agent-fh7PV65rcr


#### Check Deployment Status

Let's wait for the deployment to complete:


In [6]:
import time

# Wait for the agent to be ready
status_response = agentcore_runtime.status()
status = status_response.endpoint["status"]

end_status = ["READY", "CREATE_FAILED", "DELETE_FAILED", "UPDATE_FAILED"]
while status not in end_status:
    print(f"Waiting for deployment... Current status: {status}")
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint["status"]

print(f"Final status: {status}")

Retrieved Bedrock AgentCore status for: customer_support_agent


Final status: READY


### Step 4: Invoking Your Deployed Agent

Now that our agent is deployed and ready, let's test it with some queries. We invoke the agent with the right authorization token type. In out case it'll be Cognito access token. Copy the access token from the cell above

<div style="text-align:left"> 
    <img src="images/invoke.png" width="100%"/> 
</div>

#### Using the AgentCore Starter Toolkit

We can validate that the agent works using the AgentCore Starter Toolkit for invocation. The starter toolkit can automatically create a session id for us to query our agent. Alternatively, you can also pass the session id as a parameter during invocation. For demonstration purpose, we will create our own session id.

In [11]:
import uuid

# Create a session ID for demonstrating session continuity
session_id = uuid.uuid4()

# Test different customer support scenarios
# user_query = "My Iphone is not connecting with the Bluetooth. What should I do?"
user_query = "제 아이폰이 블루투스와 연결되지 않아요. 어떻게 해야 하나요?"

bearer_token = reauthenticate_user(
    cognito_config.get("client_id"), 
    cognito_config.get("client_secret")
)

response = agentcore_runtime.invoke(
    {"prompt": user_query}, 
    bearer_token=bearer_token,
    session_id=str(session_id)
)
response

Invoking BedrockAgentCore agent 'customer_support_agent' via cloud endpoint


{'response': '"아이폰의 블루투스 연결 문제를 해결하기 위한 몇 가지 단계를 안내해 드리겠습니다:\\n\\n### 기본 문제 해결 단계:\\n\\n1. **블루투스 켜고 끄기**:\\n   - 설정 앱 → 블루투스를 끈 다음 몇 초 후 다시 켜보세요.\\n   - 컨트롤 센터에서 블루투스 아이콘을 눌러 끈 후 다시 활성화해 보세요.\\n\\n2. **기기 재부팅**:\\n   - 아이폰을 완전히 종료한 후 다시 켜보세요.\\n\\n3. **네트워크 설정 초기화**:\\n   - 설정 → 일반 → 재설정 → 네트워크 설정 재설정\\n   - (참고: 이 방법은 저장된 Wi-Fi 암호와 블루투스 연결을 모두 삭제합니다)\\n\\n4. **페어링된 장치 삭제 후 다시 연결**:\\n   - 설정 → 블루투스 → 연결하려는 장치 옆의 (i) 아이콘 → \\"이 장치 지우기\\"\\n   - 그런 다음 장치를 다시 페어링해 보세요.\\n\\n5. **최신 iOS 업데이트 확인**:\\n   - 설정 → 일반 → 소프트웨어 업데이트에서 최신 iOS 버전 확인\\n\\n6. **연결하려는 블루투스 장치도 재부팅**:\\n   - 헤드폰, 스피커 등 연결하려는 장치도 껐다 켜보세요.\\n\\n### 지속적인 문제 발생 시:\\n\\n- 다른 블루투스 장치와 연결을 시도하여 문제가 특정 장치에만 있는지 확인해 보세요.\\n- 안전 모드에서 아이폰 재부팅을 시도해 보세요.\\n- 애플 지원 센터에 문의하시면 더 전문적인 도움을 받으실 수 있습니다.\\n\\n혹시 특정 블루투스 장치와의 연결 문제인지, 아니면 모든 블루투스 장치에서 발생하는 문제인지 알려주시면 더 구체적인 도움을 드릴 수 있을 것 같습니다. 다른 질문이 있으시거나 추가 지원이 필요하시면 언제든지 말씀해 주세요!"'}

#### Invoking the agent with session continuity

Since we are using AgentCore Runtime, we can easily continue our conversation with the same session id.

In [9]:
# user_query = "I've turned my Bluetooth on and off but it still does not work"
user_query = "블루투스를 켰다 껐다 해봤는데도 여전히 작동하지 않아요"
response = agentcore_runtime.invoke(
    {"prompt": user_query}, 
    bearer_token=bearer_token,
    session_id=str(session_id)
)
response

Invoking BedrockAgentCore agent 'customer_support_agent' via cloud endpoint


{'response': '"아이폰 블루투스 문제에 대해 계속 불편을 겪고 계시는 것 같습니다. 한국어로 도움을 드리겠습니다.\\n\\n더 심화된 문제 해결 방법을 안내해 드리겠습니다:\\n\\n1. **네트워크 설정 초기화**:\\n   - 설정 > 일반 > 재설정 > 네트워크 설정 재설정\\n   - 참고: 저장된 모든 Wi-Fi 네트워크와 비밀번호, 셀룰러 설정 및 VPN 구성이 삭제됩니다\\n\\n2. **강제 재시작**:\\n   - iPhone 8 이상: 음량 올리기 버튼을 눌렀다 뗀 후, 음량 내리기 버튼을 눌렀다 뗀 다음, 측면 버튼을 Apple 로고가 나타날 때까지 길게 누르세요\\n   - iPhone 7: 음량 내리기 버튼과 전원 버튼을 Apple 로고가 나타날 때까지 함께 길게 누르세요\\n   - iPhone 6s 이하: 홈 버튼과 전원 버튼을 Apple 로고가 나타날 때까지 함께 길게 누르세요\\n\\n3. **iOS 업데이트 확인**:\\n   - 설정 > 일반 > 소프트웨어 업데이트\\n   - 연결성 문제에 대한 버그 수정이 포함되어 있을 수 있으니 가능한 업데이트가 있으면 설치하세요\\n\\n4. **기기 설정 초기화**:\\n   - 설정 > 일반 > 재설정 > 모든 설정 재설정\\n   - 참고: 개인 데이터는 유지되지만 모든 시스템 설정이 기본값으로 돌아갑니다\\n\\n5. **간섭 확인**:\\n   - 전자레인지나 다른 블루투스 기기와 같은 간섭을 일으킬 수 있는 장치가 근처에 없는지 확인하세요\\n\\n위의 방법들로도 문제가 해결되지 않는다면, 하드웨어 문제일 수 있습니다. 아이폰은 1년 제조사 보증이 제공되므로, 기기가 아직 보증 기간 내라면 수리나 교체를 받으실 수 있습니다.\\n\\n추가 도움이 필요하시거나 더 자세한 정보를 원하시면 알려주세요. 문제 해결을 위해 계속 도와드리겠습니다."'}

#### Invoking the agent with a new user
In the example below we have not mentioned the Iphone device in the second query, but our agent still has the context of it. This is due to the AgentCore Runtime session continuity. The agent won't know the context for a new user.

In [10]:
# Creating a new session ID for demonstrating new customer
session_id2 = uuid.uuid4()

# user_query = "Still not working. What is going on?"
user_query = "여전히 작동하지 않아요. 무슨 일이 일어나고 있는 거죠?"
response = agentcore_runtime.invoke(
    {"prompt": user_query}, 
    bearer_token=bearer_token,
    session_id=str(session_id2)
)
response

Invoking BedrockAgentCore agent 'customer_support_agent' via cloud endpoint


{'response': '"안녕하세요. 죄송합니다만, 어떤 기기의 문제인지 정확히 파악하기 어렵습니다. 제공해주신 정보에 따르면 iPhone의 블루투스 연결 문제이거나 게임 콘솔에 관한 문제일 수 있습니다. 도움을 드리기 위해 좀 더 자세한 정보가 필요합니다.\\n\\n혹시 iPhone의 블루투스 연결 문제에 대해 말씀하시는 것인가요? 아니면 Gaming Console Pro에 관련된 문제인가요? 또한 어떤 기본적인 문제해결 단계를 이미 시도해보셨는지 알려주시면 더 정확한 도움을 드릴 수 있습니다.\\n\\n문제를 해결하기 위해 스마트폰 또는 게임 콘솔에 대한 제품 정보를 확인해 드릴 수 있습니다. 정확히 어떤 기기에 대한 도움이 필요하신지 알려주시겠어요?"'}

In this case our agent does not have the context anymore and needs more information. 

And it is all it takes to have a secure and scalable endpoint for our Agent with no need to manage all the underlying infrastructure!

### Step 5: AgentCore Observability

[AgentCore Observability](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/observability.html) provides monitoring and tracing capabilities for AI agents using Amazon OpenTelemetry Python Instrumentation and Amazon CloudWatch GenAI Observability.

#### Agents

Default AgentCore Runtime configuration allows for logging our agent's traces on CloudWatch by means of **AgentCore Observability**. These traces can be seen on the AWS CloudWatch GenAI Observability dashboard. Navigate to Cloudwatch &rarr; GenAI Observability &rarr; Bedrock AgentCore.

![Agents Overview on CloudWatch](images/observability_agents.png)

#### Sessions

The Sessions view shows the list of all the sessions associated with all agents in your account.

![sessions](images/sessions_lab5_observability.png)

#### Traces

Trace view lists all traces from your agents in this account. To work with traces:

- Choose Filter traces to search for specific traces.
- Sort by column name to organize results.
- Under Actions, select Logs Insights to refine your search by querying across your log and span data or select Export selected traces to export.

![traces](images/traces_lab4_observability.png)


### Congratulations! 🎉

You have successfully completed **Lab 4: Deploy to Production - Use AgentCore Runtime with Observability!**

Here is what you accomplished:

##### Production-Ready Deployment:

- Prepared your agent for production with minimal code changes (only 4 lines added)
- Validated proper session isolation between different customers
- Confirmed session continuity + memory persistence and context awareness per session

##### Enterprise-Grade Security & Identity:

- Implemented secure authentication using Cognito integration with JWT tokens
- Configured proper IAM roles and execution permissions for production workloads
- Established identity-based access control for secure agent invocation

##### Comprehensive Observability:

- Enabled AgentCore Observability for full request tracing across all customer sessions
- Configured CloudWatch GenAI Observability dashboard monitoring

##### Current Limitations (We'll fix these next!):

- **Developer Focused Interaction** - Agent accessible via SDK/API calls but no user-friendly web interface
- **Manual Session Management** - Requires programmatic session creation rather than intuitive user experience

##### Next Up [Lab 4: Build User Interface →](lab-05-frontend.ipynb)
In Lab 5, you'll complete the customer experience by building a user-friendly interface !! Lets go !!
